# Claycode SIGGRAPH Evaluation

In [33]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go


df = pd.read_csv('results.csv')

df['type'] = df['filename'].str.split('-').map(lambda x: "-".join(x[:-1]).replace("_clay", "clay"))
df['square_area_perc'] = df['square_dimension_perc']**2

df = df[['successful', 'square_area_perc', 'type']]

# Turn the square side dimension into an area percentage
grouped = df.groupby(['square_area_perc', 'type'])

# Aggregate the 'successful' column
df = grouped['successful'].agg(['sum', 'count']).reset_index()

# Calculate success rate
df['success_rate'] = df['sum'] / df['count']

In [34]:
def hex_to_rgb(hex_color):
    """Convert a hexadecimal color code to an RGB tuple."""
    hex_color = hex_color.lstrip('#')  # Remove the '#' if it exists
    return tuple(int(hex_color[i:i+2], 16) for i in (0, 2, 4))


# Define all pairs to display
# Also defines display order
pairs_to_display = [
    ['qrart2', 'qrart'],
    ['qr2', 'qr'],
    ['claycode2', 'claycode'],
    ['claycode2-fragment', 'claycode-fragment'],
    ['claycodeart2', 'claycodeart'],
    ['barcode2', 'barcode']
]

name_mapping = {
    'claycode': 'Claycode (R=1)',
    'claycode-fragment': 'Claycode (R=2)',
    'qr': 'QR',
    'qrart': 'Styl. QR',
    'claycodeart': 'Styl. Claycode (R=2)',
    'barcode': 'Barcode',
    'claycode2': 'Claycode (R=1)',
    'claycodeart2': 'Styl. Claycode (R=2)',
    'claycode2-fragment': 'Claycode (R=2)',
    'qr2': 'QR',
    'qrart2': 'Styl. QR',
    'barcode2': 'Barcode'
}

##################
# Colors
##################

claycode_color_main = '#156b59'  
claycode_color_variant2 = '#298c73'  
claycode_color_variant1 = '#26a78a' 
qr_color_main = '#d95f02'  # Main shade for qr
qr_color_variant = '#f5861a'  # orange for variant
barcode_color = '#1160b8' 

color_scheme = {
    'claycode': claycode_color_main, 
    'claycodeart': claycode_color_variant1, 
    'claycode-fragment': claycode_color_variant2, 
    'qr': qr_color_main, 
    'qrart': qr_color_variant, 
    'barcode': barcode_color,
    'claycode2': claycode_color_main, 
    'claycodeart2': claycode_color_variant1, 
    'claycode2-fragment': claycode_color_variant2, 
    'qr2': qr_color_main, 
    'qrart2': qr_color_variant, 
    'barcode2': barcode_color
}

##################
# Line dashes
##################

line_dash_map = {
    'claycode': 'dot', 
    'claycodeart': 'dot', 
    'claycode-fragment': 'dot', 
    'qr': 'dot', 
    'qrart': 'dot', 
    'barcode': 'dot',
    'claycode2': 'solid', 
    'claycodeart2': 'solid', 
    'claycode2-fragment': 'solid', 
    'qr2': 'solid', 
    'qrart2': 'solid', 
    'barcode2': 'solid'
}

##################
# Marker styles
##################

marker_qr = 'square'
marker_qrart = 'diamond'
marker_claycode = 'circle'
marker_claycodeart = 'bowtie'
marker_claycode_fragment = 'diamond-tall'
marker_barcode = 'x'

marker_map = {
    'qr': marker_qr,
    'qrart': marker_qrart,
    'claycode': marker_claycode,
    'claycodeart': marker_claycodeart,
    'barcode': marker_barcode,
    'claycode-fragment': marker_claycode_fragment,
    'claycode2': marker_claycode,
    'claycodeart2': marker_claycodeart,
    'claycode2-fragment': marker_claycode_fragment,
    'qr2': marker_qr,
    'qrart2': marker_qrart,
    'barcode2': marker_barcode,
}


MULT = 1.2
SIZE_QR = 9
SIZE_QR_ART = 10
SIZE_CLAYCODE = 8
SIZE_CLAYCODE_ART = 12
SIZE_CLAYCODE_FRAGMENT = 10
SIZE_BARCODE = 9
marker_size_map = {
    'qr': 0,
    'qrart': 0,
    'claycode': 0,
    'claycodeart': 0,
    'barcode': 0,
    'claycode-fragment': SIZE_CLAYCODE_FRAGMENT,
    'claycode2': SIZE_CLAYCODE,
    'claycodeart2': SIZE_CLAYCODE_ART,
    'claycode2-fragment': SIZE_CLAYCODE_FRAGMENT,
    'qr2': SIZE_QR,
    'qrart2': SIZE_QR_ART,
    'barcode2': SIZE_BARCODE,
}
marker_size_map = { key: value*MULT for key,value in marker_size_map.items() }

##################
# Trace size and opacity maps
##################

trace_opacity_map = {
    'qr': 1.0,
    'qrart': 1,
    'claycode': 1,
    'claycodeart': 1,
    'barcode': 1.0,
    'claycode-fragment':1 ,
    'claycode2': 1,
    'claycodeart2': 1,
    'claycode2-fragment': 1,
    'qr2': 1.0,
    'qrart2': 1,
    'barcode2': 1.0,
}

##################
# Updated Plot
##################

fig = go.Figure()

for pair in pairs_to_display:
    # Get the data for both types in the pair
    trace_1 = df[df['type'] == pair[0]]
    trace_2 = df[df['type'] == pair[1]]
    
    fig.add_trace(go.Scatter(
        x=trace_1['square_area_perc'], 
        y=trace_1['success_rate'], 
        mode='lines+markers', 
        name=name_mapping.get(pair[0].replace("2", ""), pair[0]), 
        line=dict(
            color=color_scheme.get(pair[0], '#000000'),
            width=marker_size_map.get(pair[0], 10),
            dash=line_dash_map.get(pair[0], 'solid')
        ),
        marker=dict(
            symbol=marker_map.get(pair[0], 'circle'),
            size=marker_size_map.get(pair[0], 10),
            opacity=trace_opacity_map.get(pair[0], 1.0)
        ),
        showlegend=True
    ))

    fig.add_trace(go.Scatter(
        x=trace_2['square_area_perc'], 
        y=trace_2['success_rate'], 
        mode='lines+markers', 
        name=name_mapping.get(pair[1], pair[1]),
        line=dict(
            color=f"rgba({','.join(map(str, hex_to_rgb(color_scheme.get(pair[1], '#000000'))))}, 0.2)",
            dash=line_dash_map.get(pair[1], 'dot')
        ),
        marker=dict(
            symbol=marker_map.get(pair[1], 'circle'),
            size=marker_size_map.get(pair[1], 10),
            opacity=0
        ),
        showlegend=False
    ))

    # Add shaded areas as before
    for i in range(len(trace_1) - 1):
        x_val_1 = trace_1['square_area_perc'].iloc[i]
        x_val_2 = trace_1['square_area_perc'].iloc[i + 1]
        
        y_val_1_1 = trace_1['success_rate'].iloc[i]
        y_val_2_1 = trace_2[trace_2['square_area_perc'] == x_val_1]['success_rate'].iloc[0]
        
        y_val_1_2 = trace_1['success_rate'].iloc[i + 1]
        y_val_2_2 = trace_2[trace_2['square_area_perc'] == x_val_2]['success_rate'].iloc[0]
        
        fill_color = 'rgba(0, 255, 0, 0.1)' if y_val_2_1 <= y_val_1_1 and y_val_2_2 <= y_val_1_2 else 'rgba(255, 0, 0, 0.1)'
        
        fig.add_trace(go.Scatter(
            x=[x_val_1, x_val_2, x_val_2, x_val_1], 
            y=[y_val_1_1, y_val_1_2, y_val_2_2, y_val_2_1],
            fill='toself', fillcolor=fill_color, mode='none',
            showlegend=False, line=dict(color='rgba(0, 0, 0, 0)')
        ))

FIG_MULT = 0.8
fig.update_layout(
    xaxis_title="Area Occluded (%)",
    height=410*FIG_MULT,
    width=615*FIG_MULT,
    margin=dict(t=10, b=10, l=10, r=10),
    legend=dict(
        x=0.57, 
        y=0.9, 
        xanchor='left', 
        yanchor='top',
        bgcolor='rgba(255, 255, 255, 0.7)',
        bordercolor='black', 
        borderwidth=1
    )
)

fig.update_traces(opacity=1, line=dict(width=2.3))
fig.update_yaxes(title=None)

fig.update_xaxes(range=[0.001, 0.26], tickformat='.0%', tickvals=[.01, .04, .09, .16, .25])
fig.update_yaxes(range=[-0.05, 1.05], tickformat='.0%')

fig.write_image("occlusion-deformation.pdf", format="pdf", engine="kaleido")
fig.show()
